In [ ]:
# Colab에 Khaiii 설치

import os
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python


Cloning into 'khaiii'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 957 (delta 25), reused 49 (delta 19), pack-reused 877
Receiving objects: 100% (957/957), 33.03 MiB | 27.28 MiB/s, done.
Resolving deltas: 100% (383/383), done.
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-

In [ ]:
# src에 있는 preanal.manual 파일 dir로 옮기는 Code

import shutil

filename = "preanal.manual"
src = '/content/'
dir = '/content/khaiii/rsc/src/'

shutil.move(src+filename, dir+filename)

'/content/khaiii/rsc/src/preanal.manual'

In [ ]:
# Khaiii 사용자 사전 추가
''' 
** Khaiii 사용자 사전 추가를 참고  **
'''

!cd /content/khaiii/rsc
!mkdir -p /content/build/share/khaiii
!PYTHONPATH=/content/khaiii/src/main/python /content/khaiii/rsc/bin/compile_preanal.py --rsc-src=/content/khaiii/rsc/src --rsc-dir=/content/build/share/khaiii

INFO:root:preanal.auto
INFO:root:preanal.manual
ERROR:root:preanal.manual:24: [DUPLICATED] with "식빵	식빵/NNG": "식빵*	식빵/NNP"
ERROR:root:preanal.manual:29: [DUPLICATED] with "커피	커피/NNG": "커피*	커피/NNP"
ERROR:root:preanal.manual:51: [DUPLICATED] with "인절미	인절미/NNG": "인절미*	인절미/NNP"
ERROR:root:preanal.manual:52: [DUPLICATED] with "딸기	딸기/NNG": "딸기*	딸기/NNP"
ERROR:root:preanal.manual:57: [DUPLICATED] with "음료	음료/NNG": "음료*	음료/NNP"
ERROR:root:preanal.manual:125: [DUPLICATED] with "케이크	케이크/NNG": "케이크*	케이크/NNP"
ERROR:root:preanal.manual:145: [DUPLICATED] with "우유	우유/NNG": "우유*	우유/NNP"
ERROR:root:preanal.manual:165: [DUPLICATED] with "주문	주문/NNG": "주문*	주문/NNP"
ERROR:root:preanal.manual:166: [DUPLICATED] with "제작	제작/NNG": "제작*	제작/NNP"
ERROR:root:preanal.manual:181: [DUPLICATED] with "파인애플*	파인애플/NNG": "파인애플*	파인애플/NNP"
ERROR:root:preanal.manual:229: [DUPLICATED] with "레몬	레몬/NNG": "레몬*	레몬/NNP"
ERROR:root:preanal.manual:262: [DUPLICATED] with "초콜릿	초콜릿/NNG": "초콜릿*	초콜릿/NNP"
ERROR:root:preanal.manual:311: [DUPL

## Gensim Topic Modeling


In [ ]:
import pandas as pd


data = pd.read_csv('/content/final_input.csv', encoding='utf-8').drop(['Unnamed: 0'], axis=1)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1566 entries, 0 to 1565
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Nickname  1566 non-null   object
 1   Review    1566 non-null   object
dtypes: object(2)
memory usage: 24.6+ KB


# Gensim LDA를 위한 데이터 전처리
## Experiment 1) Khaiii에서 명사, 어근만 추출해 Tokenizing




In [ ]:
from khaiii import KhaiiiApi
api = KhaiiiApi(rsc_dir="/content/build/share/khaiii")

n_tags = ['NNG', 'NNP', 'NNB', 'XR']#, 'VV', "VA"] # 동사도 넣고 싶으면 추가

'''
input : 추출할 Review의 list ;
output : n_tags의 tag와 일치하는 text list ; 
'''

def extract_corpus_khaiii(texts):
    extract_corpus = []
    for line in texts:
      if str(line) != 'nan':
        nouns = []

        for word in api.analyze(str(line)):
          for morphs in word.morphs:
            if morphs.tag in n_tags:
              if len(morphs.lex) > 1:
                nouns.append(morphs.lex)
              else:
                continue

        extract_corpus.append(nouns)

    return extract_corpus
  

In [ ]:
# 네이버 영수증 리뷰, 인스타 리뷰에서 명사, 어근을 추출
khaiii_xr = extract_corpus_khaiii(data['Review']) 

#khaiii_all = extract_corpus_khaiii(data['Review']) # 명사, 동사, 형용사, 어근 모두 추출

In [ ]:
# 추출 전 데이터와 비교

print(len(data), len(khaiii_xr))
#print(len(data), len(khaiii_all))

1566 1566


In [ ]:
import gensim
from gensim.utils import simple_preprocess
# Build the bigram and trigram models
bigram = gensim.models.Phrases(khaiii_xr, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[khaiii_xr], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
# remove Stopwords, make bigram, trigram function

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [ ]:
# Stop words 제거 및 trigram 만들기
stop_words = ['점심', '아침', '종류', '오랜만', '자리', '직원', '아이', '오픈', '날씨', '저녁', '따뜻', '메뉴', '다양','창녕', '달성군', '동대문', '수성', '영등포', '뻐용', '용산구', '관악', '오늘', '달서', '강동구', '모두', '평택', '과천', '해운대구', '합천', '계양', '매장', '광진구', '강동', '기장군', '남해', '서초', '경산시', '광양시', '논산시', '서대문', '단양군', '아산', '동구', '구미', '칠곡군', '공주시', '부산진', '계룡', '청도군', '제천', '부안', '정도', '하동군', '부여군', '괴산', '여수시', '송파구', '문경시', '거제시', '옹진', '함평', '평택시', '하동', '복기', '제주', '영주', '고령군', '울주군', '여수', '동해시', '아메리카', '보령시', '화성', '동두천', '원주', '정읍시', '커피', '영암', '양산', '철원군', '군위', '고흥군', '양구군', '강진', '여주시', '의정부시', '강화', '서대문구', '영광군', '저번', '북구', '시흥시', '양산시', '영양', '계양구', '사하구', '장수', '홍천군', '목포', '사람', '봉화군', '완주', '순천시', '경주시', '수성구', '이드', '완도', '구례', '보성군', '카페', '제천시', '카페라', '하남', '무안', '용산', '정선군', '울릉군', '안성시', '무주', '통영시', '울진군', '광진', '화성시', '영덕', '의정부', '마포구', '충주시', '기장', '괴산군', '익산', '완도군', '금천구', '성주', '김제시', '서산', '속초', '사상구', '남해군', '영천시', '가평군', '인제군', '제주시', '과천시', '동대문구', '광명', '구로구', '논산', '담양군', '오산시', '장성', '서천', '화순군', '양평군', '광산구', '성동', '중랑구', '이천시', '방문', '관악구', '무안군', '예천군', '남원', '함안', '구리', '성북구', '군포', '양천', '영덕군', '강화군', '평창', '김해시', '해남', '울릉', '금산', '광산', '함양', '곡성군', '봉화', '울주', '보은', '장수군', '홍천', '동작구', '부천', '디저트', '순창군', '최고', '추천', '부평구', '화순', '강남구', '증평군', '영월', '부여', '진안', '철원', '영월군', '양천구', '청양군', '의왕시', '양주', '포천시', '카페예', '공주', '강서구', '호점', '거창군', '영도구', '때문', '합천군', '강북구', '속초시', '울진', '경주', '대체', '시흥', '군산', '충주', '청송', '고흥', '송파', '동래구', '단양', '횡성', '양구', '증평', '보령', '안동시', '이천', '포천', '영광', '연수', '전체', '동해', '가격', '진도군', '달성', '양평', '사천시', '청양', '원주시', '김포시', '양주시', '진천군', '남원시', '그동안', '중랑', '강릉', '순천', '이후', '고창군', '하남시', '불구', '맛집', '안성', '산청', '강진군', '청도', '삼척시', '김천시', '종로구', '담양', '진주시', '횡성군', '구례군', '파주', '광명시', '은평구', '음성', '영주시', '예산군', '고성군', '노원', '김천', '광양', '장성군', '신안군', '도봉', '분위기', '추입', '사상', '강북', '서초구', '사하', '커피랑', '김제', '임실', '고창', '영등포구', '진천', '성동구', '산청군', '달서구', '사장', '연천', '장흥', '뻐요', '유성', '주문', '의령', '아메리카노랑', '당진', '부평', '도봉구', '서천군', '밀양', '상주시', '예천', '메리카노', '부산진구', '춘천시', '상주', '함평군', '의왕', '경산', '구요', '거창', '홍성군', '마포', '신발', '금천', '대덕', '고령', '종로', '해남군', '느낌', '남양주', '부안군', '통영', '이즈니버터', '가평', '화천군', '가요', '익산시', '정보궁금팔로다음', '창녕군', '홍성', '생각', '진안군', '사천', '목포시', '옹진군', '청송군', '은평', '삼척', '연수구', '칠곡', '니당', '여주', '신안', '정선', '의령군', '연천군', '의성', '동두천시', '어제', '정읍', '요즘', '군포시', '나주', '카페입니', '내일', '함양군', '시간', '양양군', '음성군', '인제', '군산시', '유성구', '구미시', '진짜', '영암군', '광주', '태백시', '구로', '해운대', '동작', '금정구', '강남', '강서', '밀양시', '서산시', '대덕구', '강릉시', '고성', '진도', '수영', '보성', '서귀포시', '수영구', '태안군', '양양', '부천시', '태안', '태백', '완전', '의성군', '케이크', '문경', '친절', '중구', '평창군', '남구', '순창', '서귀포', '음료', '연제', '동래', '예산', '영천', '거제', '영도', '보은군', ' 굿굿굿', '함안군', '파주시', '안동', '남양주시', '노원구', '김해', '서구', '곡성', '구리시', '군위군', '임실군', '진주', '영양군', '장흥군', '광주시', '완주군', '화천', '연제구', '무주군', '오산', '성주군', '당진시', '계룡시', '춘천', '성북', '아산시', '금산군', '김포', '멍멍', '크림', '금정', '나주시']
khaiii_xr = remove_stopwords(khaiii_xr)

khaiii_xr = make_trigrams(khaiii_xr)

In [ ]:
# countvectorize를 위한 역토큰화 진행
'''
input : n_tags만 뽑힌 token list
 Ex. ['얼그레이', '마카롱', '맛']

output : 역토큰화된 detoken list
 Ex. ['얼그레이 마카롱 맛']
'''
def detokenize(token_list):
  detokenized_doc = []
  for i in range(len(token_list)):
    if token_list[i] != []:
      t = ' '.join(token_list[i])

      detokenized_doc.append(t)
  return detokenized_doc
    #detokenized_doc.append([data['Nickname'][i], t]) -> 옆에 닉네임 붙여서 내보낼거면 활성화

In [ ]:
# bigram, trigram으로 합쳐진 단어는 _로 합쳐지기 때문에 _를 제거하고 합쳐줌

for i in range(len(khaiii_xr)):
  for j in range(len(khaiii_xr[i])):
    khaiii_xr[i][j] = khaiii_xr[i][j].replace("_", "")

In [ ]:
# trigram으로 형성된 토큰 역토큰화

detoken_xr = detokenize(khaiii_xr)

## Using Gensim

In [ ]:
# install gensim

!pip install gensim


In [ ]:
# import package

from sklearn.feature_extraction.text import CountVectorizer
from gensim.matutils import Sparse2Corpus


In [ ]:

from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora


# Warning 무시
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
warnings.filterwarnings(action='ignore', category=Warning, module='gensim')
warnings.filterwarnings('ignore')

* **num_topics**: 최종 분석의 주제 수
* **passes**: 총 훈련 과정의 수. 신경망 학습에서 에포크와 같다
* **iteration** : 각 문서에 대해 업데이트를 반복하는 횟수
* **random_state**: 재현 가능한 결과를 위해 임의의 숫자를 설정한다.

### 최적의 number of words 찾기
* 지표 : Coherence

In [ ]:
vectorizer = CountVectorizer(min_df=0.0099) # CountVectorizer 생성
cv = vectorizer.fit_transform(detoken_xr)

In [ ]:
cv.shape

(1566, 1435)

In [ ]:
vectorizer = CountVectorizer() # CountVectorizer 생성

cv = vectorizer.fit_transform(detoken_xr) # fit and transform

dictionary = corpora.Dictionary([vectorizer.get_feature_names()])

corpus = Sparse2Corpus(cv.T)

lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=5,
                     random_state=42,
                     chunksize=256,
                     passes=1000,
                     per_word_topics=True,
                     iterations=500,
                     alpha=0.01,
                     eta=0.01)

coherence_lda = CoherenceModel(model=lda_model, texts=khaiii_xr, dictionary=dictionary, coherence='c_v')




In [ ]:
coherence_lda.get_coherence()

0.47489057666560325

In [ ]:
lda_model.print_topics()

[(0,
  '0.046*"딸기" + 0.040*"케익" + 0.026*"마카롱" + 0.020*"타르트" + 0.018*"감사" + 0.012*"제품" + 0.012*"준비" + 0.011*"초콜릿" + 0.011*"가지" + 0.011*"판매"'),
 (1,
  '0.022*"조합" + 0.022*"기분" + 0.020*"라떼" + 0.020*"아이스크림" + 0.016*"시원" + 0.013*"원두" + 0.012*"하루" + 0.011*"요기" + 0.010*"우유" + 0.010*"시작"'),
 (2,
  '0.021*"공간" + 0.019*"라떼" + 0.018*"주차" + 0.018*"인테리어" + 0.015*"사진" + 0.012*"아메리카노" + 0.012*"위치" + 0.011*"테이블" + 0.010*"조용" + 0.010*"내부"'),
 (3,
  '0.029*"사진" + 0.020*"음식" + 0.017*"마무리" + 0.014*"샐러드" + 0.012*"파스타" + 0.012*"서울" + 0.012*"예약" + 0.011*"샌드위치" + 0.010*"피자" + 0.010*"공간"'),
 (4,
  '0.022*"치즈" + 0.021*"스콘" + 0.019*"달달" + 0.017*"버터" + 0.015*"식감" + 0.014*"빵집" + 0.012*"부분" + 0.011*"흑임자" + 0.010*"만족" + 0.009*"고소"')]

In [ ]:
lda_model.get_topic_terms(0)

[(5275, 0.045665316),
 (23320, 0.040077545),
 (6941, 0.02603037),
 (24114, 0.020164821),
 (302, 0.018111838),
 (20117, 0.012099698),
 (20594, 0.012055655),
 (22038, 0.01093947),
 (157, 0.010904553),
 (25116, 0.010801342)]

In [ ]:
words = []
for i in range(5):
  for j in range(10):
    words.append([i,lda_model.get_topic_terms(i)[j][0], lda_model.get_topic_terms(i)[j][1]])


In [ ]:
word = pd.DataFrame(words, columns=['TopicNum', 'word', 'valnum'])

for i in range(len(word)):
  id = word['word'][i]
  token = dictionary.id2token[id]
  word['word'][i] = token
  

In [ ]:
i1 = [0, 10, 20, 30, 40]

for idx in i1:
  word['valnum'][idx:idx+10] = (word['valnum'][idx:idx+10] / word['valnum'][idx:idx+10].max()) * 100
  print(idx, idx+10)

0 10
10 20
20 30
30 40
40 50


In [ ]:
word['TopicNum'] = word['TopicNum'] + 1

In [ ]:
word

,TopicNum,word,valnum
0,1,딸기,100.000000
1,1,케익,90.510247
2,1,마카롱,58.887071
3,1,타르트,43.324731
4,1,조합,38.760512
5,1,감사,36.249078
6,1,치즈,31.881592
7,1,제품,28.820379
8,1,준비,28.523161
9,1,가지,27.178144


In [ ]:
(word['valnum'][40:50] / word['valnum'][40:50].max()) *100

40    100.000000
41     94.730896
42     69.686595
43     65.038614
44     63.644311
45     61.814120
46     55.560602
47     49.090036
48     47.980681
49     47.363129
Name: valnum, dtype: float64

In [ ]:
from openpyxl import Workbook

write_wb = Workbook()
write_ws = write_wb.active

write_ws.append(word.keys().tolist())
for i in range(len(word)):
  write_ws.append(word.loc[i].tolist())


write_wb.save('words.xlsx')

In [ ]:
# lda_model word를 추출

words = []

topic = lda_model.print_topics()
for i in range(len(data)):
  temp = []
  for j in range(5):
    temp.append(top[i][j][1])
  words.append(temp)

In [ ]:
matrix = []

top = lda_model.get_document_topics(corpus, minimum_probability=0)
for i in range(len(data)):
  temp = []
  for j in range(5):
    temp.append(top[i][j][1])
  matrix.append(temp)

In [ ]:
matrix = pd.DataFrame(matrix, columns=['Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic5'])
lda_model.get_document_topics(corpus)[0]

[(0, 0.71224284),
 (1, 0.0003992016),
 (2, 0.18306771),
 (3, 0.10389108),
 (4, 0.0003992016)]

In [ ]:
matrix

,Topic 1,Topic 2,Topic 3,Topic 4,Topic5
0,0.712245,0.000399,0.183067,0.103896,0.000399
1,0.231511,0.000250,0.527220,0.000250,0.240776
2,0.000094,0.030263,0.199611,0.615966,0.154067
3,0.232730,0.130594,0.477930,0.108043,0.050700
4,0.433059,0.000175,0.543901,0.022687,0.000175
...,...,...,...,...,...
1561,0.338594,0.000623,0.469695,0.089443,0.101616
1562,0.211881,0.158447,0.522456,0.059987,0.047229
1563,0.095146,0.130880,0.108902,0.501258,0.163814
1564,0.222112,0.098869,0.215870,0.317248,0.145900


In [ ]:
lda_model.get_document_topics(corpus, minimum_probability=0)[0]

[(0, 0.7122424),
 (1, 0.0003992016),
 (2, 0.18307014),
 (3, 0.10388902),
 (4, 0.0003992016)]

In [ ]:
username = data[1455:].drop(['Review'], axis=1)
username = username.reset_index(drop=True)
username

,Nickname
0,_______ds.pic
1,_cafelisa_
2,_im_cafe
3,_lim_cafe_
4,a_dcafeartgram
...,...
106,vnyouu
107,wonju.tour
108,yamstagram_always
109,yezzi_mon


In [ ]:
atelier = matrix[:1455]
reviewer = matrix[1455:]
reviewer = reviewer.reset_index(drop=True)

In [ ]:
reviewer.columns = [1, 2, 3, 4, 5]

In [ ]:
reviewer['topicNum'] = reviewer.idxmax(axis=1)

In [ ]:
reviewer = reviewer.drop([1, 2, 3, 4, 5], axis=1)

In [ ]:
sunsoo = pd.read_csv('sunsoo.csv', encoding='euc-kr')

In [ ]:
sunsoo

,GYEAR,PCODE,NAME,T_ID,POSITION,AGE_VA,MONEY
0,2020,50030,소형준,KT,투,19,2700만원
1,2020,50036,이강준,KT,투,19,2700만원
2,2020,50040,데스파이네,KT,투,33,450000달러
3,2020,50054,천성호,KT,내,23,2700만원
4,2020,50066,강현우,KT,포,19,2700만원
...,...,...,...,...,...,...,...
622,2020,79705,김회성,HH,내,35,6500만원
623,2020,79764,장민재,HH,투,30,11000만원
624,2020,79825,여건욱,LG,투,34,7000만원
625,2020,79847,김태훈,SK,투,30,24000만원


In [ ]:
import random

rant = []
while len(rant) < 111:
  i = random.randint(0, len(sunsoo)-1)
  if i not in rant:
    rant.append(i)
  else:
    continue

name = []

for i in rant:
  name.append(sunsoo['NAME'][i])


In [ ]:
user_sim

,userId,t1,t2,t3,t4,t5
0,_______ds.pic,0.174434,0.100756,0.347726,0.302078,0.074985
1,_cafelisa_,0.001105,0.001105,0.867983,0.128701,0.001105
2,_im_cafe,0.199941,0.123289,0.361932,0.218531,0.096310
3,_lim_cafe_,0.433371,0.060450,0.187430,0.138650,0.180049
4,a_dcafeartgram,0.242320,0.159748,0.347182,0.133152,0.117598
...,...,...,...,...,...,...
106,vnyouu,0.338594,0.000623,0.469695,0.089443,0.101616
107,wonju.tour,0.211881,0.158447,0.522456,0.059987,0.047229
108,yamstagram_always,0.095146,0.130880,0.108902,0.501258,0.163814
109,yezzi_mon,0.222112,0.098869,0.215870,0.317248,0.145900


In [ ]:
users = pd.concat([user_sim[['userId']], reviewer], axis=1)

In [ ]:
users['userName'] = name

In [ ]:
users = users[['userId', 'userName', 'topicNum']]

In [ ]:
users

,userId,userName,topicNum
0,_______ds.pic,장원삼,3
1,_cafelisa_,문선재,3
2,_im_cafe,최승준,3
3,_lim_cafe_,홍성무,1
4,a_dcafeartgram,이정용,3
...,...,...,...
106,vnyouu,박민호,3
107,wonju.tour,조한욱,3
108,yamstagram_always,윤규진,4
109,yezzi_mon,이승진,4


In [ ]:
from openpyxl import Workbook

write_wb = Workbook()
write_ws = write_wb.active

write_ws.append(users.keys().tolist())
for i in range(len(users)):
  write_ws.append(users.loc[i].tolist())


write_wb.save('users.xlsx')

In [ ]:
user_sim = pd.concat([username, reviewer], axis=1)
user_sim.columns = ['userId', 't1', 't2', 't3', 't4', 't5']

In [ ]:
user_sim

,userId,t1,t2,t3,t4,t5
0,_______ds.pic,0.174434,0.100756,0.347726,0.302078,0.074985
1,_cafelisa_,0.001105,0.001105,0.867983,0.128701,0.001105
2,_im_cafe,0.199941,0.123289,0.361932,0.218531,0.096310
3,_lim_cafe_,0.433371,0.060450,0.187430,0.138650,0.180049
4,a_dcafeartgram,0.242320,0.159748,0.347182,0.133152,0.117598
...,...,...,...,...,...,...
106,vnyouu,0.338594,0.000623,0.469695,0.089443,0.101616
107,wonju.tour,0.211881,0.158447,0.522456,0.059987,0.047229
108,yamstagram_always,0.095146,0.130880,0.108902,0.501258,0.163814
109,yezzi_mon,0.222112,0.098869,0.215870,0.317248,0.145900


In [ ]:
from openpyxl import Workbook

write_wb = Workbook()
write_ws = write_wb.active

write_ws.append(user_sim.keys().tolist())
for i in range(len(user_sim)):
  write_ws.append(user_sim.loc[i].tolist())


write_wb.save('user_sims.xlsx')

In [ ]:
from openpyxl import load_workbook

desc = []

load_wb = load_workbook("description.xlsx", data_only=True)
load_ws = load_wb['아뜰리에_이름설명키워드']

#총 1513개
for i in range(2, 1515):

    a = load_ws.cell(row=i,column=1).value
    b = load_ws.cell(row=i,column=2).value
    desc.append([a, b])
    

In [ ]:
desc =  pd.DataFrame(desc, columns=['StoreName', 'Description'])

desc = desc.drop_duplicates(['StoreName'])
desc = desc.reset_index(drop=True)

In [ ]:
desc

,StoreName,Description
0,자매상회,정성스럽게 만든 수제 마카롱 매장 입니다.\n마카롱은 수~일 12시에나옵니다.(화요...
1,카페사계,"편안한 분위기의 카페, 사계 입니다."
2,하쿠나마타타,커피에 사용되는 시럽부터 시그니처음료까지 직접 만들어 제공하고 있습니다.\n하쿠나마...
3,마린데크,None
4,청년마카롱,None
...,...,...
1448,디어라이,None
1449,배스킨라빈스안산한대점,None
1450,천안옛날호두과자 안산대부도점,None
1451,커피하겐나,None


In [ ]:
idname = pd.read_csv('idname.csv', encoding='utf-8')
idname.info()


idname = idname.drop_duplicates(['StoreName'])
idname = idname.reset_index(drop=True)
idname = idname[['StoreName', 'StoreID']]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   StoreID    1564 non-null   int64 
 1   StoreName  1564 non-null   object
dtypes: int64(1), object(1)
memory usage: 24.6+ KB


In [ ]:
temp = pd.merge(idname, desc, how='left', on='StoreName')

In [ ]:
temp = temp.fillna(" ")

In [ ]:
from openpyxl import Workbook

write_wb = Workbook()
write_ws = write_wb.active

write_ws.append(temp.keys().tolist())
for i in range(len(temp)):
  write_ws.append(temp.loc[i].tolist())


write_wb.save('ateliers.xlsx')

In [ ]:
atelier.columns = [1, 2, 3, 4, 5]

In [ ]:
temp['topicNum'] = atelier.idxmax(axis=1)

In [ ]:
temp.columns = ['atelierName', "atelierId", 'description', 'topicNum' ]

In [ ]:
temp = temp[["atelierId", 'atelierName', 'topicNum', 'description']]

In [ ]:
temp

,atelierId,atelierName,topicNum,description
0,1683173237,061,1,
1,1510207557,1004-11 카페,3,
2,1405690163,161커피스튜디오,4,전메뉴 아이스 500원 추가 유모차 전용주차장 4곳 마련 테라스 있음 천원추가시 아...
3,1367564108,1894사랑채,3,
4,31238106,1984,3,책은 문화의 뿌리이자 그 결과이다.\nthe Book is the rood and ...
...,...,...,...,...
1450,35179041,희원,3,
1451,1059588955,흰다정,2,"디저트 카페 흰다정은 말차를 이용한 디저트와 빙수, 안미츠, 커피를 판매하고 있습니..."
1452,1046173103,흰별제빵소,3,"반죽, 성형, 파이, 오븐, 케이크, 샌드위치.\n파티쉐들의 정성이 담긴 작업 공정..."
1453,1548171259,흰여울비치,3,"흰여울 신상카페! 영도핫플! 비치바가 있는 인생샷명소! 호주바리스타의 맛난커피, ..."


In [ ]:
desc

,StoreName,StoreID
0,자매상회,정성스럽게 만든 수제 마카롱 매장 입니다.\n마카롱은 수~일 12시에나옵니다.(화요...
1,카페사계,"편안한 분위기의 카페, 사계 입니다."
2,하쿠나마타타,커피에 사용되는 시럽부터 시그니처음료까지 직접 만들어 제공하고 있습니다.\n하쿠나마...
3,마린데크,None
4,청년마카롱,None
...,...,...
1447,헬로에스프레소,안산 팬케이크 맛집 달달한 헬로에스프레소\n안산 고잔동 카페 :: 헬로에소프레소 수...
1448,디어라이,None
1449,배스킨라빈스안산한대점,None
1450,천안옛날호두과자 안산대부도점,None


In [ ]:
temp = pd.merge(idname, desc, on='StoreName')

In [ ]:
temp

,StoreName,StoreID_x,StoreID_y
0,061,1683173237,None
1,1004-11 카페,1510207557,None
2,161커피스튜디오,1405690163,전메뉴 아이스 500원 추가 유모차 전용주차장 4곳 마련 테라스 있음 천원추가시 아...
3,1894사랑채,1367564108,None
4,1984,31238106,책은 문화의 뿌리이자 그 결과이다.\nthe Book is the rood and ...
...,...,...,...
1504,희원,35179041,None
1505,흰다정,1059588955,"디저트 카페 흰다정은 말차를 이용한 디저트와 빙수, 안미츠, 커피를 판매하고 있습니..."
1506,흰별제빵소,1046173103,"반죽, 성형, 파이, 오븐, 케이크, 샌드위치.\n파티쉐들의 정성이 담긴 작업 공정..."
1507,흰여울비치,1548171259,"흰여울 신상카페! 영도핫플! 비치바가 있는 인생샷명소! 호주바리스타의 맛난커피, ..."


In [ ]:
desc

,StoreName,StoreID
0,자매상회,정성스럽게 만든 수제 마카롱 매장 입니다.\n마카롱은 수~일 12시에나옵니다.(화요...
1,카페사계,"편안한 분위기의 카페, 사계 입니다."
2,하쿠나마타타,커피에 사용되는 시럽부터 시그니처음료까지 직접 만들어 제공하고 있습니다.\n하쿠나마...
3,마린데크,None
4,청년마카롱,None
...,...,...
1447,헬로에스프레소,안산 팬케이크 맛집 달달한 헬로에스프레소\n안산 고잔동 카페 :: 헬로에소프레소 수...
1448,디어라이,None
1449,배스킨라빈스안산한대점,None
1450,천안옛날호두과자 안산대부도점,None


In [ ]:
idname

,StoreName,StoreID
0,061,1683173237
1,1004-11 카페,1510207557
2,161커피스튜디오,1405690163
3,1894사랑채,1367564108
4,1984,31238106
...,...,...
1450,희원,35179041
1451,흰다정,1059588955
1452,흰별제빵소,1046173103
1453,흰여울비치,1548171259


In [ ]:
name = data[:1455].drop(['Review'], axis=1)
name.columns = ["StoreName"]
name.reset_index(drop=True)

,StoreName
0,061
1,1004-11 카페
2,161커피스튜디오
3,1894사랑채
4,1984
...,...
1450,희원
1451,흰다정
1452,흰별제빵소
1453,흰여울비치


In [ ]:
atelier_idname = pd.merge(name, idname, on='StoreName', how='inner')

In [ ]:
atelier_idname = atelier_idname[['StoreID', 'StoreName']]

In [ ]:
atelier_idname[['StoreID']]

,StoreID
0,1683173237
1,1510207557
2,1405690163
3,1367564108
4,31238106
...,...
1450,35179041
1451,1059588955
1452,1046173103
1453,1548171259


In [ ]:
temp = pd.concat([atelier_idname[['StoreID']],atelier], axis=1) 

In [ ]:
temp.columns=['atelierId', 't1', 't2', 't3', 't4', 't5']

In [ ]:
temp

,atelierId,t1,t2,t3,t4,t5
0,1683173237,0.712243,0.000399,0.183058,0.103889,0.000399
1,1510207557,0.231511,0.000250,0.527220,0.000250,0.240774
2,1405690163,0.000094,0.030265,0.199610,0.615965,0.154067
3,1367564108,0.232730,0.130593,0.477934,0.108043,0.050701
4,31238106,0.433059,0.000175,0.543903,0.022689,0.000175
...,...,...,...,...,...,...
1450,35179041,0.056442,0.092491,0.822358,0.000178,0.028537
1451,1059588955,0.049199,0.331015,0.271969,0.158433,0.189387
1452,1046173103,0.054042,0.101001,0.480496,0.026699,0.337762
1453,1548171259,0.142587,0.070029,0.662463,0.049843,0.075078


In [ ]:
from openpyxl import Workbook

write_wb = Workbook()
write_ws = write_wb.active

write_ws.append(temp.keys().tolist())
for i in range(len(temp)):
  write_ws.append(temp.loc[i].tolist())


write_wb.save('temp.xlsx')